In [1]:
from numpy import array
from pickle import dump
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# load doc into memory
def load_sequences(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [3]:
# load
in_filename = 'data/trump_sequences.txt'
doc = load_sequences(in_filename)
lines = doc.split('\n')

In [3]:


# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

# define model
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_length),
    LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True),
    LSTM(256, dropout=0.2, recurrent_dropout=0.2),
    Dense(256, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            515250    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 256)           314368    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dense_1 (Dense)              (None, 10305)             2648385   
Total params: 4,069,107
Trainable params: 4,069,107
Non-trainable params: 0
_________________________________________________________________


In [4]:
# early stop callback
early_stop = [EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')]

# fit model
model.fit(X, y, batch_size=128, epochs=100, callbacks=early_stop, validation_split=0.1)

# save the model to file
model.save('models/trump_model.h5')

# save the tokenizer
dump(tokenizer, open('data/tokenizer.pkl', 'wb'))

Train on 148068 samples, validate on 16453 samples
Epoch 1/100
148068/148068 [==============================] - 144s 973us/sample - loss: 6.7365 - val_loss: 6.6627
Epoch 2/100
148068/148068 [==============================] - 138s 933us/sample - loss: 6.3868 - val_loss: 6.5295
Epoch 3/100
148068/148068 [==============================] - 136s 921us/sample - loss: 6.2510 - val_loss: 6.4791
Epoch 4/100
148068/148068 [==============================] - 137s 925us/sample - loss: 6.1555 - val_loss: 6.3998
Epoch 5/100
148068/148068 [==============================] - 136s 921us/sample - loss: 6.0805 - val_loss: 6.3704
Epoch 6/100
148068/148068 [==============================] - 136s 919us/sample - loss: 6.0237 - val_loss: 6.3475
Epoch 7/100
148068/148068 [==============================] - 136s 921us/sample - loss: 5.9490 - val_loss: 6.3004
Epoch 8/100
148068/148068 [==============================] - 145s 977us/sample - loss: 5.8745 - val_loss: 6.2634
Epoch 9/100
148068/148068 [==================